<a href="https://colab.research.google.com/github/ashwinvaswani/Generative-Modelling-of-Images-from-Speech/blob/master/src/keras/keras_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import pickle
import numpy as np
import pandas as pd

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D,AvgPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,BatchNormalization,LeakyReLU,MaxPool2D
from keras.models import Sequential
from keras.layers import Conv2DTranspose,Reshape

In [0]:
PATH = './drive/My Drive/TIP/Dataset/'
PATH_TO_MAIN = './drive/My Drive/TIP/'
YT_LINK = 'www.youtube.com/watch?v='

In [0]:
with open(PATH_TO_MAIN + 'Pickles/encoder_trainX.pkl','rb') as f:
    x_enc_train = pickle.load(f)

In [0]:
with open(PATH_TO_MAIN + 'Pickles/encoder_trainY.pkl','rb') as f:
    y_train_encoder= pickle.load(f)

In [0]:
x_enc_train.shape

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(x_enc_train,y_train_encoder,test_size = 0.05)

In [0]:
X_train.shape

In [0]:
model_encoder = Sequential()
model_encoder.add(Conv2D(64, kernel_size=(4,4),strides=(1,1),  input_shape=[598,257,2],use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Conv2D(64, kernel_size=(4,4),strides=(1,1), use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Conv2D(128, kernel_size=(4,4),strides=(1,1), use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(MaxPool2D(pool_size=(2,1)))

model_encoder.add(Conv2D(128, kernel_size=(4,4),strides=(1,1), use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(MaxPool2D(pool_size=(2,1)))

model_encoder.add(Conv2D(256, kernel_size=(4,4),strides=(1,1), use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(MaxPool2D(pool_size=(2,1)))

model_encoder.add(Conv2D(512, kernel_size=(4,4),strides=(1,1), use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())
 
model_encoder.add(Conv2D(512, kernel_size=(4,4),strides=(2,2), use_bias=False))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Conv2D(512, kernel_size=(4,4),strides=(2,2), use_bias=False))

model_encoder.add(AvgPool2D(pool_size=(15,1),strides = (1,1)))
model_encoder.add(LeakyReLU(alpha=0.1))
model_encoder.add(BatchNormalization())

model_encoder.add(Flatten())
model_encoder.add(Dense(4096,activation='relu'))
model_encoder.add(Dropout(0.2))

model_encoder.add(Dense(2048,activation='relu'))

model_encoder.summary()


In [0]:
model_encoder.compile(loss='mse',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [0]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
 
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
    tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    cp_callback
]
 
 
model_encoder.fit(X_train, y_train, batch_size=2,validation_data=(X_val, y_val), epochs=10 ,verbose=1)

In [0]:
y_pred_encoder = model_encoder.predict(y_train_encoder)
with open(PATH_TO_MAIN + 'Pickles/y_pred_encoder.pkl','wb') as f:
    pickle.dump(y_pred_encoder,f)

In [0]:
model_encoder.save(PATH_TO_MAIN + 'Models/Encoder1.h5')